In [2]:
import os
from datasets import load_dataset
from huggingface_hub import login

HF_LOGIN_CODE = os.getenv("HF_LOGIN_CODE")
login(HF_LOGIN_CODE)

dataset = load_dataset("aegean-ai/ai-lectures-spring-24")

In [3]:
# Persist to local folder (will create indexes, arrow files, etc.)
dataset.save_to_disk("./raw_original_video_dataset")

Saving the dataset (0/5 shards):   0%|          | 0/8 [00:00<?, ? examples/s]